## Preparation

### Installation

We assume that the repo is cloned, all necessary packages are installed, including calling the script:

```./install_packages.sh```

and the code is compiled:

```./build.sh```

### Changing directory to the repo root

In [1]:
cd ../..

/Users/leo/SourceTreeGit/FlexNeuART.refact2021


### Downloading demo data

In [2]:
!wget boytsov.info/datasets/flecsneurt-demo-2021-02-08.tar.bz2

--2021-06-25 16:30:58--  http://boytsov.info/datasets/flecsneurt-demo-2021-02-08.tar.bz2
Resolving boytsov.info (boytsov.info)... 69.60.127.165
Connecting to boytsov.info (boytsov.info)|69.60.127.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7511061 (7.2M) [application/x-bzip2]
Saving to: ‘flecsneurt-demo-2021-02-08.tar.bz2’

flecsneurt-demo-202 100%[===================>]   7.16M  2.40MB/s    in 3.0s    

2021-06-25 16:31:01 (2.40 MB/s) - ‘flecsneurt-demo-2021-02-08.tar.bz2’ saved [7511061/7511061]



In [3]:
# Unpacking it
!tar jxvf flecsneurt-demo-2021-02-08.tar.bz2

x collections/squad/
x collections/squad/exper_desc/
x collections/squad/input_data/
x collections/squad/input_data/train_bitext/
x collections/squad/input_data/dev1/
x collections/squad/input_data/test/
x collections/squad/input_data/dev2/
x collections/squad/input_data/train/
x collections/squad/input_data/train/AnswerFields.jsonl
x collections/squad/input_data/train/QuestionFields.jsonl
x collections/squad/input_data/train/qrels.txt
x collections/squad/input_data/dev2/AnswerFields.jsonl
x collections/squad/input_data/dev2/QuestionFields.jsonl
x collections/squad/input_data/dev2/qrels.txt
x collections/squad/input_data/test/AnswerFields.jsonl
x collections/squad/input_data/test/QuestionFields.jsonl
x collections/squad/input_data/test/qrels.txt
x collections/squad/input_data/dev1/AnswerFields.jsonl
x collections/squad/input_data/dev1/QuestionFields.jsonl
x collections/squad/input_data/dev1/qrels.txt
x collections/squad/input_data/train_bitext/AnswerFields.jsonl
x collections/squad/inp

In [4]:
# Creating a Lucene index
!scripts/index/create_lucene_index.sh squad

Using collection root: collections
Data directory:    collections/squad/input_data
Index directory:   collections/squad/lucene_index
Index field name:  text
Exact match param: 
Removing previously created index (if exists)
Checking input sub-directory: dev1
Found indexable data file: dev1/AnswerFields.jsonl
Checking input sub-directory: dev2
Found indexable data file: dev2/AnswerFields.jsonl
Checking input sub-directory: test
Found indexable data file: test/AnswerFields.jsonl
Checking input sub-directory: train
Found indexable data file: train/AnswerFields.jsonl
Checking input sub-directory: train_bitext
Found indexable data file: train_bitext/AnswerFields.jsonl
Found query file: dev1/QuestionFields.jsonl
Found query file: dev2/QuestionFields.jsonl
Found query file: test/QuestionFields.jsonl
Found query file: train/QuestionFields.jsonl
Found query file: train_bitext/QuestionFields.jsonl
Using the data input file: AnswerFields.jsonl
JAVA_OPTS=-Xms8388608k -Xmx14680064k -server
[main] IN

In [5]:
#!creating a forward index for two fields:
# text is a parsed text field
# text_raw is a raw text field that keeps the text as is
# -clean removes all previous forward indices
!scripts/index/create_fwd_index.sh squad mapdb  \
                               "text:parsedText text_unlemm:raw" \
                               -clean

Using collection root: collections
Data directory:            collections/squad/input_data
Forward index directory:   collections/squad/forward_index/
Clean old index?:          1
Removing previously created index (if exists)
Field list definition:     text:parsedText text_unlemm:raw
Checking input sub-directory: dev1
Found indexable data file: dev1/AnswerFields.jsonl
Checking input sub-directory: dev2
Found indexable data file: dev2/AnswerFields.jsonl
Checking input sub-directory: test
Found indexable data file: test/AnswerFields.jsonl
Checking input sub-directory: train
Found indexable data file: train/AnswerFields.jsonl
Checking input sub-directory: train_bitext
Found indexable data file: train_bitext/AnswerFields.jsonl
Found query file: dev1/QuestionFields.jsonl
Found query file: dev2/QuestionFields.jsonl
Found query file: test/QuestionFields.jsonl
Found query file: train/QuestionFields.jsonl
Found query file: train_bitext/QuestionFields.jsonl
JAVA_OPTS=-Xms12582912k -Xmx14680064k 

## API demo

In [6]:
from scripts.py_flexneuart.setup import *

In [7]:
# add Java JAR to the class path
configure_classpath('target')

In [8]:
# create a resource manager
resource_manager=create_featextr_resource_manager('collections/squad/forward_index')

### Retrieval

In [9]:
from scripts.config import QUESTION_FILE_JSON, QREL_FILE, DOCID_FIELD, TEXT_FIELD_NAME

In [10]:
from scripts.py_flexneuart.cand_provider import *
# create a candidate provider/generator
cand_prov = create_cand_provider(resource_manager, PROVIDER_TYPE_LUCENE, 'collections/squad/lucene_index')

In [11]:
QUERY_TEXT = "university notre dame student run"

In [12]:
# An example of running a text query
query_res = run_text_query(cand_prov, 20, QUERY_TEXT)
query_res

(1022,
 [CandidateEntry(doc_id='@4309', score=12.375007629394531),
  CandidateEntry(doc_id='@4323', score=12.343233108520508),
  CandidateEntry(doc_id='@2608', score=12.092323303222656),
  CandidateEntry(doc_id='@4310', score=11.984010696411133),
  CandidateEntry(doc_id='@4303', score=11.907716751098633),
  CandidateEntry(doc_id='@4350', score=11.397300720214844),
  CandidateEntry(doc_id='@4346', score=11.215911865234375),
  CandidateEntry(doc_id='@4316', score=11.073210716247559),
  CandidateEntry(doc_id='@4336', score=10.876322746276855),
  CandidateEntry(doc_id='@4345', score=10.455368041992188),
  CandidateEntry(doc_id='@4320', score=10.438504219055176),
  CandidateEntry(doc_id='@2607', score=10.252201080322266),
  CandidateEntry(doc_id='@4325', score=10.212079048156738),
  CandidateEntry(doc_id='@4330', score=9.874446868896484),
  CandidateEntry(doc_id='@4348', score=9.816712379455566),
  CandidateEntry(doc_id='@4321', score=9.762029647827148),
  CandidateEntry(doc_id='@4338', sco

In [13]:
# An example of running a generic query interface
query_res = run_query(cand_prov, 20, {TEXT_FIELD_NAME : QUERY_TEXT}, default_query_id=FAKE_QUERY_ID)
query_res

(1022,
 [CandidateEntry(doc_id='@4309', score=12.375007629394531),
  CandidateEntry(doc_id='@4323', score=12.343233108520508),
  CandidateEntry(doc_id='@2608', score=12.092323303222656),
  CandidateEntry(doc_id='@4310', score=11.984010696411133),
  CandidateEntry(doc_id='@4303', score=11.907716751098633),
  CandidateEntry(doc_id='@4350', score=11.397300720214844),
  CandidateEntry(doc_id='@4346', score=11.215911865234375),
  CandidateEntry(doc_id='@4316', score=11.073210716247559),
  CandidateEntry(doc_id='@4336', score=10.876322746276855),
  CandidateEntry(doc_id='@4345', score=10.455368041992188),
  CandidateEntry(doc_id='@4320', score=10.438504219055176),
  CandidateEntry(doc_id='@2607', score=10.252201080322266),
  CandidateEntry(doc_id='@4325', score=10.212079048156738),
  CandidateEntry(doc_id='@4330', score=9.874446868896484),
  CandidateEntry(doc_id='@4348', score=9.816712379455566),
  CandidateEntry(doc_id='@4321', score=9.762029647827148),
  CandidateEntry(doc_id='@4338', sco

In [14]:
# An example of running a generic query interface
query_res = run_query(cand_prov, 20, {DOCID_FIELD: FAKE_QUERY_ID, TEXT_FIELD_NAME : QUERY_TEXT})
query_res

(1022,
 [CandidateEntry(doc_id='@4309', score=12.375007629394531),
  CandidateEntry(doc_id='@4323', score=12.343233108520508),
  CandidateEntry(doc_id='@2608', score=12.092323303222656),
  CandidateEntry(doc_id='@4310', score=11.984010696411133),
  CandidateEntry(doc_id='@4303', score=11.907716751098633),
  CandidateEntry(doc_id='@4350', score=11.397300720214844),
  CandidateEntry(doc_id='@4346', score=11.215911865234375),
  CandidateEntry(doc_id='@4316', score=11.073210716247559),
  CandidateEntry(doc_id='@4336', score=10.876322746276855),
  CandidateEntry(doc_id='@4345', score=10.455368041992188),
  CandidateEntry(doc_id='@4320', score=10.438504219055176),
  CandidateEntry(doc_id='@2607', score=10.252201080322266),
  CandidateEntry(doc_id='@4325', score=10.212079048156738),
  CandidateEntry(doc_id='@4330', score=9.874446868896484),
  CandidateEntry(doc_id='@4348', score=9.816712379455566),
  CandidateEntry(doc_id='@4321', score=9.762029647827148),
  CandidateEntry(doc_id='@4338', sco

### Forward index demo

In [15]:
from scripts.py_flexneuart.fwd_index import get_forward_index

#### First let's play with a raw index that keeps ony unparsed text

In [16]:
raw_indx = get_forward_index(resource_manager, 'text_unlemm')

In [17]:
# Index type
raw_indx.indx_fld_type

'textRaw'

In [18]:
raw_indx.get_doc_text_raw('@4302')

'architecturally school catholic character atop main building gold dome golden statue virgin mary immediately main building facing copper statue christ arms upraised legend venite ad omnes main building basilica sacred heart immediately basilica grotto marian place prayer reflection replica grotto lourdes france virgin mary reputedly appeared saint bernadette soubirous 1858 end main drive direct line connects 3 statues gold dome simple modern stone statue mary'

#### A parsed index has more info

In [19]:
parsed_indx = get_forward_index(resource_manager, 'text')

In [20]:
# Index type
parsed_indx.indx_fld_type

'parsedText'

In [21]:
parsed_indx.get_doc_parsed('@4302')

DocEntryParsed(word_ids=[1, 470, 480, 549, 770, 848, 857, 867, 1143, 1193, 1291, 1514, 1562, 1597, 1897, 2210, 2425, 2513, 2579, 3171, 3207, 3357, 3806, 3899, 3960, 4056, 4334, 4790, 5881, 6258, 6274, 6629, 6645, 7051, 7557, 8066, 9139, 10063, 11826, 12878, 13240, 16221, 20752, 32578, 32579, 32580, 32581, 32582, 32583], word_qtys=[1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 4, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1], word_id_seq=[10063, 1, 6274, 848, 8066, 1291, 1193, 6645, 9139, 3171, 11826, 1143, 4334, 1597, 1291, 1193, 2210, 3899, 11826, 2425, 3806, 32578, 857, 32579, 7051, 13240, 1291, 1193, 4790, 7557, 4056, 1597, 4790, 32580, 6258, 1897, 5881, 16221, 6629, 32580, 32581, 3207, 1143, 4334, 20752, 470, 2579, 32582, 32583, 12878, 1562, 1291, 3357, 867, 770, 2513, 549, 11826, 6645, 9139, 1514, 480, 3960, 11826, 4334], doc_len=65)

In [22]:
# Let's extract the first document word and its info
parsed_indx.get_word_by_id(10063), parsed_indx.get_word_entry_by_id(10063)

('architecturally', WordEntry(word_id=10063, word_freq=11))

### Ranker API demo

### First, we run two experiments that involve training a model

#### Delete old results if they are present

In [23]:
!rm -rf collections/squad/results/dev1/feat_exper

####  Running two experiments (you can add ``-no_separate_shell`` to print logs to the screen for debug purposes)

In [24]:
!scripts/exper/run_experiments.sh squad \
    exper_desc/exper_list.json \
    -train_part train \
    -test_part dev1 \
    -test_cand_qty_list 100

Using collection root: collections
The number of CPU cores:      8
The number of || experiments: 1
The number of threads:        8
Experiment descriptor file:                                 collections/squad/exper_desc/exper_list.json
Default test set:                                           dev1
Number of parallel experiments:                             1
Number of threads in feature extractors/query applications: 8
Parsed experiment parameters:
experSubdir:feat_exper/bm25=text
extrType:exper_desc/extractors/bm25=text.json
testOnly:0
Started a process 17182, working dir: collections/squad/results/dev1/feat_exper/bm25=text
Process log file: collections/squad/results/dev1/feat_exper/bm25=text/exper.log
Waiting for 1 child processes
Process with pid=17182 finished successfully.
Parsed experiment parameters:
experSubdir:feat_exper/bm25=text+cosine=text
extrType:exper_desc/extractors/bm25=text+cosine=text.json
testOnly:0
Started a process 17248, working dir: collections/squad/results/d

#### Print results generated by ``trec_eval``

In [25]:
!cat collections/squad/results/dev1/feat_exper/bm25\=text+cosine\=text/rep/out_100.rep 

# of queries:    2448
NDCG@10:  0.925700
NDCG@20:  0.927800
NDCG@100: 0.929900
P20:      0.048600
MAP:      0.914100
MRR:      0.914100
Recall:   0.983660


### Second, we can use this model to re-rank and evaluate results using Python API

#### A basic example

In [26]:
from scripts.py_flexneuart.ranker import *
from scripts.py_flexneuart.utils import *

In [27]:
MODEL_FILE_NAME='collections/squad/results/dev1/feat_exper/bm25=text+cosine=text/letor/out_squad_train_20.model'
FEAT_EXTR_FILE_NAME='collections/squad/exper_desc/extractors/bm25=text+cosine=text.json'
QUERY_FILE_NAME=f'collections/squad/input_data/dev1/{QUESTION_FILE_JSON}'
QREL_FILE_NAME=f'collections/squad/input_data/dev1/{QREL_FILE}'

#### A list of experimental descriptors, which in turn reference descriptors for feature extractors

In [28]:
!cat collections/squad/exper_desc/exper_list.json

[
  {
    "experSubdir": "feat_exper/bm25=text",
    "extrType" : "exper_desc/extractors/bm25=text.json",
    "testOnly": 0
  },
  {
    "experSubdir": "feat_exper/bm25=text+cosine=text",
    "extrType" : "exper_desc/extractors/bm25=text+cosine=text.json",
    "testOnly": 0
  }
]


#### A (two-feature) feature extractor confguration, which is used in the second experiment

In [29]:
!cat collections/squad/exper_desc/extractors/bm25=text+cosine=text.json

{
"extractors" : [
 {
  "type" : "TFIDFSimilarity",
  "params" : {
    "indexFieldName" : "text",
    "similType" : "bm25",
    "k1"        : "1.2",
    "b"         : "0.75"
  }
 },
 {
  "type" : "TFIDFSimilarity",
  "params" : {
    "indexFieldName" : "text",
    "similType" : "cosine"
  }
 }
]
}


#### A simple linear model trained to combine feature scores produced by the feature-extractor

In [30]:
!cat collections/squad/results/dev1/feat_exper/bm25=text+cosine=text/letor/out_squad_train_20.model

## Coordinate Ascent
## Restart = 10
## MaxIteration = 50
## StepBase = 0.05
## StepScale = 2.0
## Tolerance = 0.001
## Regularized = false
## Slack = 0.001
1:0.5464353855587661 2:-0.4535646144412339

#### A toy example where we generate a list of candidates for merely one query (using the candidate provider) and re-rank them using the Java-layer re-ranker

In [31]:
java_ranker = JavaQueryRanker(resource_manager, feat_extr_file_name=FEAT_EXTR_FILE_NAME, model_file_name=MODEL_FILE_NAME)

In [32]:
query_dict = {DOCID_FIELD : FAKE_QUERY_ID, 
              TEXT_FIELD_NAME : QUERY_TEXT}
java_ranker.rank_candidates(query_res[1], query_dict)

[('@4323', 0.5067933451547831),
 ('@4303', 0.48961460832303094),
 ('@4309', 0.47475528548967094),
 ('@2608', 0.4549869297008977),
 ('@4346', 0.4413701458990109),
 ('@4316', 0.4355512672038647),
 ('@4310', 0.4332624225820959),
 ('@4320', 0.4277601854466731),
 ('@2067', 0.4229184488598747),
 ('@4342', 0.4052451627386573),
 ('@4307', 0.39438917386601124),
 ('@4336', 0.3835355648995015),
 ('@4321', 0.37744164995227447),
 ('@4350', 0.36827643790295006),
 ('@4325', 0.3681315865508774),
 ('@4330', 0.3626696414510794),
 ('@2607', 0.3619121883629782),
 ('@4345', 0.3617851826918027),
 ('@4348', 0.3596303010435613),
 ('@4338', 0.3456784985640041)]

#### There's a function (used only for evaluation) to score candidates without sorting them scores

In [33]:
java_ranker.score_candidates(query_res[1], query_dict)

{'@4309': 0.47475528548967094,
 '@4323': 0.5067933451547831,
 '@2608': 0.4549869297008977,
 '@4310': 0.4332624225820959,
 '@4303': 0.48961460832303094,
 '@4350': 0.36827643790295006,
 '@4346': 0.4413701458990109,
 '@4316': 0.4355512672038647,
 '@4336': 0.3835355648995015,
 '@4345': 0.3617851826918027,
 '@4320': 0.4277601854466731,
 '@2607': 0.3619121883629782,
 '@4325': 0.3681315865508774,
 '@4330': 0.3626696414510794,
 '@4348': 0.3596303010435613,
 '@4321': 0.37744164995227447,
 '@4338': 0.3456784985640041,
 '@4307': 0.39438917386601124,
 '@2067': 0.4229184488598747,
 '@4342': 0.4052451627386573}

#### A toy example where we re-rank the list of candidate using a BERT re-ranker

In [34]:
!wget boytsov.info/models/msmarco_doc/2019/bert_vanilla/model.best

--2021-06-25 16:36:28--  http://boytsov.info/models/msmarco_doc/2019/bert_vanilla/model.best
Resolving boytsov.info (boytsov.info)... 69.60.127.165
Connecting to boytsov.info (boytsov.info)|69.60.127.165|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 438863972 (419M) [text/plain]
Saving to: ‘model.best’

model.best          100%[===================>] 418.53M  11.0MB/s    in 50s     

2021-06-25 16:37:18 (8.43 MB/s) - ‘model.best’ saved [438863972/438863972]



In [35]:
# Re-ranking on CPU, which can be fairly slow
cedr_ranker = PythonNNQueryRanker(resource_manager, 
                         query_field_name='text_unlemm', max_query_len=32,
                         index_field_name='text_unlemm', max_doc_len = 512 - 32 - 3,
                         device_name='cpu', batch_size=4, model_file_name='model.best')

/Users/leo/anaconda3/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/leo/anaconda3/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/leo/anaconda3/lib/python3.8/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing th

In [36]:
query_dict = {DOCID_FIELD : FAKE_QUERY_ID, 
              'text_unlemm' : QUERY_TEXT}
cedr_ranker.rank_candidates(query_res[1], query_dict)

[('@4303', 3.6410412788391113),
 ('@4316', 2.2296292781829834),
 ('@4309', 1.0998224020004272),
 ('@4310', 0.9482794404029846),
 ('@4338', 0.8910526037216187),
 ('@4330', 0.7861899137496948),
 ('@4323', 0.5350112915039062),
 ('@2608', 0.5348656177520752),
 ('@4336', 0.28653576970100403),
 ('@4348', 0.19181787967681885),
 ('@4325', 0.1362343430519104),
 ('@2607', 0.13104315102100372),
 ('@4342', 0.08876024186611176),
 ('@4307', 0.07658989727497101),
 ('@4350', 0.029552193358540535),
 ('@2067', -0.24131406843662262),
 ('@4320', -0.3115100860595703),
 ('@4345', -0.3343726396560669),
 ('@4346', -0.45976755023002625),
 ('@4321', -0.4623507857322693)]

#### A comprehensive example where we evaluate **all** queries from `dev1`

In [37]:
from scripts.data_convert.convert_common import *
all_queries = read_queries(QUERY_FILE_NAME)

In [38]:
# Query sample
all_queries[0:5]

[{'DOCNO': '10595',
  'text_unlemm': 'beyonce lighter skin color costuming',
  'text': 'beyonce lighter skin color costume'},
 {'DOCNO': '10608',
  'text_unlemm': 'exclusion social political groups targets genocide cppcg legal',
  'text': 'exclusion social political group target genocide cppcg legal'},
 {'DOCNO': '10575',
  'text_unlemm': 'beyonce giselle knowles-carter',
  'text': 'beyonce giselle knowles-carter'},
 {'DOCNO': '10570',
  'text_unlemm': 'school architecture',
  'text': 'school architecture'},
 {'DOCNO': '10576',
  'text_unlemm': 'bee-yon-say born september 4 1981 american',
  'text': 'bee-yon-say bear september 4 1981 american'}]

In [39]:
from tqdm import tqdm
TOP_K=100

run_dict = {}
for query_dict in tqdm(all_queries):
    qid = query_dict[DOCID_FIELD]
    query_res = run_text_query(cand_prov, TOP_K, query_dict[TEXT_FIELD_NAME])
    rank_res = java_ranker.score_candidates(query_res[1], query_dict)
    run_dict[qid] = rank_res

100%|██████████| 2448/2448 [00:17<00:00, 136.87it/s]


#### Finally, let us compute various metrics using our Python code. Note that results should match results previously produced by `trec_eval`

In [40]:
from scripts.eval_common import *
qrels=read_qrels_dict(QREL_FILE_NAME)

In [41]:
for eval_obj in [NormalizedDiscountedCumulativeGain(10), \
                 NormalizedDiscountedCumulativeGain(20), \
                 MeanAveragePrecision(), \
                 MeanReciprocalRank()]:
    print(eval_run(rerank_run=run_dict, metric_func=eval_obj, qrels_dict=qrels))

0.9236950321010553
0.9258372608440565
0.9120987070833969
0.9120987070833969


#### Optionally we can save the run to be later evaluated using external evaluation tools

In [42]:
write_run_dict(run_dict, 'run.txt')

In [43]:
!head run.txt

10595 Q0 @2069 1 0.5034178698142255 fake_run
10595 Q0 @17442 2 0.19901719649322208 fake_run
10595 Q0 @9111 3 0.19225397364078217 fake_run
10595 Q0 @1769 4 0.19119073815505058 fake_run
10595 Q0 @2228 5 0.19033063864976146 fake_run
10595 Q0 @8585 6 0.18989114401672533 fake_run
10595 Q0 @18222 7 0.18815638945124924 fake_run
10595 Q0 @9121 8 0.1853920865272263 fake_run
10595 Q0 @18223 9 0.18299288131025784 fake_run
10595 Q0 @9120 10 0.1778561648387402 fake_run
